# 创建自己的图结构

In [10]:
from typing import List, Mapping, Any, Set


class Graph:
    def __init__(self):
        # key 是该节点的唯一标识。 key 等于待转换的图结构中节点的表示方式, 比如其他图中节点的表示方式是 1,2,3, 或者 'A', 'B', 'C', 那么 key 就是对应的值
        # value 是具体的节点。 value 就是我们利用待转换的图结构中的节点, 所创建出来的属于我们自己结构的节点。
        self.nodes: Mapping[Any, Node] = {}
        self.edges: Set[Edge] = set()


class Node:
    def __init__(self, val):
        self.val = val
        self.ind = 0
        self.outd = 0
        self.nexts: List[Node] = []
        self.edges: List[Edge] = []


class Edge:
    def __init__(self, fr: Node, to: Node, weight=0):
        self.fr = fr
        self.to = to
        self.weight = weight


In [ ]:
""" 支持的结构:
matrix = [
    [权值, fr节点, to节点], # 单向边: 
]
"""
def generate_graph2(matrix: List[List[int]]):
    graph = Graph()
    for item in matrix: # 每一个 item 都是一条边
        weight, fr, to = item # 
        # 判断边的两个节点是否已存在, 不存在则新增
        if fr not in graph.nodes:
            graph.nodes[fr] = Node(fr)
        if to not in graph.nodes:
            graph.nodes[to] = Node(to)
        # 不管节点存不存在, 新的边肯定是要添加的, 节点的属性也是要更新的
        fr_node = graph.nodes.get(fr)
        to_node = graph.nodes.get(to)
        new_edge = Edge(fr_node, to_node, weight)
        graph.edges.add(new_edge) # 新的边
        fr_node.edges.append(new_edge) # fr 增加一个邻接边
        fr_node.nexts.append(to_node) # fr 增加一个邻接点
        fr_node.outd += 1 # fr 的出度 + 1
        to_node.ind += 1  # to 的入度 + 1
    return graph

In [11]:
""" 支持的结构:
matrix = [
    [节点, 节点], # 双向边
]
"""
def generate_graph(matrix):
    graph = Graph()
    for one_edge in matrix:
        n1, n2 = one_edge # 双向边
        # 添加新节点
        if n1 not in graph.nodes:
            graph.nodes[n1] = Node(n1)
        if n2 not in graph.nodes:
            graph.nodes[n2] = Node(n2)
        n1_node, n2_node = graph.nodes.get(n1), graph.nodes.get(n2)
        e1, e2 = Edge(n1_node, n2_node), Edge(n2_node, n1_node)
        # 添加边
        graph.edges.add(e1)
        graph.edges.add(e2)
        n1_node.edges.append(e1)
        n2_node.edges.append(e2)
        # 更新节点属性
        n1_node.nexts.append(n2_node)
        n2_node.nexts.append(n1_node)
        n1_node.outd += 1
        n1_node.ind += 1
        n2_node.outd += 1
        n2_node.ind += 1
    return graph

# TEST - 广度优先遍历 BFS

In [12]:
from queue import Queue


# 广度有限遍历, 先处理好当前节点的所有邻接点, 再去处理邻接点的邻接点。 已经处理过的节点不要再处理。
# 利用 queue 保存待处理的节点, 利用 set 保存已经处理过的节点
def bfs(node):
    if node is None:
        return
    queue = Queue()
    selected_node = set()
    queue.put(node)
    selected_node.add(node)
    while not queue.empty():
        n = queue.get()
        print(n.val, end=' ')  # 出队列时处理
        # 处理完后, 先处理该节点的所有邻接点, 所以将他们先全部添加到队列中
        for wait_node in n.nexts:
            # 处理过的节点不要再处理
            if wait_node not in selected_node:
                selected_node.add(wait_node)
                queue.put(wait_node)


In [13]:
matrix = [
    ['A', 'B'],
    ['A', 'C'],
    ['A', 'D'],
    ['B', 'C'],
    ['D', 'C'],
    ['E', 'C'],
    ['D', 'F'],
]
graph = generate_graph(matrix)
bfs(graph.nodes.get('E'))

E C A B D F 

# TEST - 深度优先遍历 DFS

In [14]:
# 深度优先, 一直逮着一个往下处理, 没得处理了才回退(出栈)
def dfs(node):
    stack = []
    selected_node = set()
    stack.append(node)
    print(node.val, end=' ')  # 处理时刻
    selected_node.add(node)  # 处理后标记起来

    while 0 != len(stack):  # stack 为空说明都被处理过了
        n = stack.pop()  # 将处理过的拿走。
        # 逮着 n 继续往下处理
        for next_node in n.nexts:
            if next_node not in selected_node:  # 逮着一个没处理过的
                stack.append(n)  # 此时还无法确定 n 的 nexts 都处理过了, 所以重新将 n 压栈
                stack.append(next_node)  # 拿到 stack 等待处理
                print(next_node.val, end=' ')  # 处理时刻
                selected_node.add(next_node)  # 处理后标记
                break  # 逮着一个就马上处理, 这才是 "深度优先"


In [17]:
matrix = [
    ['A', 'B'],
    ['A', 'C'],
    ['B', 'D'],
    ['B', 'E'],
]
graph = generate_graph(matrix)
dfs(graph.nodes.get('A'))

A B D E C 

# TEST - 拓扑排序

# TEST - 最小生成树 MST

# TEST - Dijkstra 算法 - 最短路径